In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
import os

In [2]:
# ① 데이터 읽어오기
train_file = os.path.join('./models/intent', "Intent_train_data.csv")
df = pd.read_csv(train_file, delimiter=',', header=None)
df = df[[0, 1, 2]]

In [3]:
df.columns = ['query', 'intent', 'intent_info']
seq_len = list(map(lambda x : len(x.split(' ')), df['query']))
max(seq_len)  # 문장 최대길이

8

In [4]:
# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN = max(seq_len)

def GlobalParams():
    global MAX_SEQ_LEN

In [5]:
# '질문(query)' 과 '의도(intent)'
queries = df['query'].tolist()
intents = df['intent'].tolist()

In [6]:
queries[:20]

['사이드메뉴',
 '사이드 메뉴',
 '빵종류',
 '종류',
 '음료종류',
 '커피종류',
 '식사종류',
 '에이드종류',
 '사이드메뉴?',
 '사이드 메뉴?',
 '빵종류?',
 '종류?',
 '음료종류?',
 '커피종류?',
 '식사종류?',
 '에이드종류?',
 '사이드메뉴는 뭐 있어?',
 '사이드 메뉴는 뭐 있어?',
 '빵종류는 뭐 있어?',
 '종류는 뭐 있어?']

In [7]:
intents[:20]
os.path

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [35]:
# 전처리 모듈 불러오기
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [47]:
from transformers import ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator") # 전처리 모듈 불러오기

sequences = []
for sentence in queries:
    pos = tokenizer.tokenize(sentence)
    seq = tokenizer.convert_tokens_to_ids(pos)
    sequences.append(seq)

TypeError: 'module' object is not callable

In [10]:
# 단어 시퀀스 생성 (가장 시간 많이 걸림)
# 해당 단어에 매칭되는 번호로 시퀀스 생성

# ★ 시간 제법 걸림 * 몇십초 정도..

# sequences = []
# for sentence in queries:
#     pos = p.pos(sentence)
#     keywords = p.get_keywords(pos, without_tag=True)
#     seq = p.get_wordidx_sequence(keywords)
#     sequences.append(seq)

In [10]:
sequences

[[7837, 67],
 [7837, 67],
 [1],
 [8941],
 [1],
 [1],
 [1],
 [1, 8941],
 [7837, 67],
 [7837, 67],
 [1],
 [8941],
 [1],
 [1],
 [1],
 [1, 8941],
 [7837, 67, 552, 25],
 [7837, 67, 552, 25],
 [1, 552, 25],
 [8941, 552, 25],
 [1, 552, 25],
 [1, 552, 25],
 [1, 552, 25],
 [1, 8941, 552, 25],
 [7837, 67],
 [7837, 67],
 [1],
 [8941],
 [1],
 [1],
 [1],
 [1, 8941],
 [7837, 67, 42, 21],
 [7837, 67, 42, 21],
 [1, 42, 21],
 [8941, 42, 21],
 [1, 42, 21],
 [1, 42, 21],
 [1, 42, 21],
 [1, 8941, 42, 21],
 [7837, 67, 42, 21],
 [7837, 67, 42, 21],
 [1, 42, 21],
 [8941, 42, 21],
 [1, 42, 21],
 [1, 42, 21],
 [1, 42, 21],
 [1, 8941, 42, 21],
 [7837, 67, 25],
 [7837, 67, 25],
 [1, 25],
 [8941, 25],
 [1, 25],
 [1, 25],
 [1, 25],
 [1, 8941, 25],
 [7837, 67, 25],
 [7837, 67, 25],
 [1, 25],
 [8941, 25],
 [1, 25],
 [1, 25],
 [1, 25],
 [1, 8941, 25],
 [7837, 67, 552, 25],
 [7837, 67, 552, 25],
 [1, 552, 25],
 [8941, 552, 25],
 [1, 552, 25],
 [1, 552, 25],
 [1, 552, 25],
 [1, 8941, 552, 25],
 [7837, 67, 627, 8],
 [78

In [11]:
# ② 단어 인덱스 시퀀스 벡터 
# 단어 시퀀스 벡터 크기 (MAX_SEQ_LEN 로 동일하게 맞추기, 패딩처리)
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [12]:
padded_seqs

array([[13422,  4503,  5082, ...,     0,     0,     0],
       [13422,  8297,     0, ...,     0,     0,     0],
       [ 2836,  4317,  4488, ...,     0,     0,     0],
       ...,
       [ 8567,     0,     0, ...,     0,     0,     0],
       [ 8567,     0,     0, ...,     0,     0,     0],
       [ 8567,     0,     0, ...,     0,     0,     0]])

In [17]:
tokenizer.vocab_size

35000

In [18]:
# ③ 학습용, 검증용, 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents)) # 패딩처리된 시퀀스와 의도(intent) 리스트 전체를 데이터셋 객체로
ds = ds.shuffle(len(queries), seed=12) # 랜덤 섞기

# 학습셋:검셋:테스트셋 = 7:2:1
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(64)
val_ds = ds.skip(train_size).take(val_size).batch(64)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(64)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 30
VOCAB_SIZE = tokenizer.vocab_size + 1 #전체 단어 개수

In [19]:
# ④ CNN 모델 정의
# keras 함수형 모델 방식으로 구현
input_layer = Input(shape=(MAX_SEQ_LEN,))  # 입력크기
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(12, name='logits')(dropout_hidden)  # 최종적으로 12가의 의도 클래스를 분류. 결과로 나온 값(logits) 을을 점수(score) 라 부른다
predictions = Dense(12, activation=tf.nn.softmax)(logits)


In [20]:
# ⑤ 모델 생성 
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 8, 128)       4480128     ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 8, 128)       0           ['embedding[0][0]']              
                                                                                                  
 conv1d (Conv1D)                (None, 6, 128)       49280       ['dropout[0][0]']                
                                                                                              

In [22]:
# 모델학습
# ★ 시간 걸림 ★ 
checkpoint_cb = keras.callbacks.ModelCheckpoint('./models/intent/intent_model.h5', save_best_only=True) # 제일 좋은 모델 저장
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1,
         callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/30
182/182 [==============================] - 13s 67ms/step - loss: 0.9509 - accuracy: 0.7119 - val_loss: 0.0452 - val_accuracy: 0.9897
Epoch 2/30
182/182 [==============================] - 13s 70ms/step - loss: 0.0535 - accuracy: 0.9878 - val_loss: 0.0128 - val_accuracy: 0.9982
Epoch 3/30
182/182 [==============================] - 12s 68ms/step - loss: 0.0263 - accuracy: 0.9939 - val_loss: 0.0066 - val_accuracy: 0.9991
Epoch 4/30
182/182 [==============================] - 12s 66ms/step - loss: 0.0164 - accuracy: 0.9962 - val_loss: 0.0023 - val_accuracy: 0.9997
Epoch 5/30
182/182 [==============================] - 12s 64ms/step - loss: 0.0102 - accuracy: 0.9977 - val_loss: 0.0032 - val_accuracy: 0.9994
Epoch 6/30
182/182 [==============================] - 12s 68ms/step - loss: 0.0086 - accuracy: 0.9981 - val_loss: 0.0043 - val_accuracy: 0.9991
Epoch 7/30
182/182 [==============================] - 12s 64ms/step - loss: 0.0082 - accuracy: 0.9984 - val_loss: 0.0037 - val_accuracy:

In [26]:
# ⑦ 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

26/26 [==============================] - 0s 2ms/step - loss: 0.0030 - accuracy: 0.9994
Accuracy: 99.939686
loss: 0.003027


## 의도분류 모듈 테스트

In [27]:
from utils.Preprocess import Preprocess
from models.intent.IntentModel import IntentModel
import os

p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [28]:
intent = IntentModel(model_name=os.path.join('./models/intent', 'intent_model.h5'), preprocess=p)

In [32]:
query = "메뉴" # 개인컵, 메뉴판, 테이크아웃

In [33]:
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

In [34]:
print(query)
print('의도 예측 클래스 : ', predict)
print('의도 예측 레이블 : ', predict_label)

메뉴
의도 예측 클래스 :  11
의도 예측 레이블 :  욕설
